# Mortgage Lending Analysis

<img
    style="width:800px; height:300px; object-fit:cover;"
    src="https://free4kwallpapers.com/uploads/originals/2018/06/09/itap-of-some-more-beach-houses-wallpaper.jpg"
/>

## Introduction

This project will conduct an **exploratory data analysis** of the modified Home Mortgage Disclosure Act (HMDA) [2022 Loan Application Register](https://ffiec.cfpb.gov/documentation/2022/publications/modified-lar/) (LAR) data. 

The data contains **loan-level information filed by financial institutions** and modified by the [Consumer Financial Protection Bureau](https://www.consumerfinance.gov/) (CFPB) to protect the privacy of borrowers and lenders.

The HMDA is a United States federal law that requires financial institutions to maintain, report, and publicly disclose loan-level information about **mortgages**.

A detailed explanation of the data fields and definitions can be found in the [HMDA Documentation](https://ffiec.cfpb.gov/documentation/2022/lar-data-fields/).

In [1]:
# Print the file size in bytes and the number of lines
!stat -f "Size: %z bytes" ./data/2022_combined_mlar_header.txt && wc -l ./data/2022_combined_mlar_header.txt | awk '{print "Lines:", $1}'

Size: 3736212304 bytes
Lines: 16076740


The file is **3.7 gigabytes** in size **after decompression** and contains approximately **16 million records**.

The data can be downloaded directly from [here](https://s3.amazonaws.com/cfpb-hmda-public/prod/dynamic-data/combined-mlar/2022/header/2022_combined_mlar_header.zip) as a `.zip` file.

> **Note**: For more details on this project, please refer to its [Github repository](https://github.com/mandelbrojt/hum-dah).

## Data Wrangling

### Importing Libraries and Modules

The following libraries and modules are required for the correct execution of the code in this project:

In [29]:
import dask.dataframe as dd
import lar_schema as ls
import pandas as pd
import numpy as np
import plotly.express as px

### Selecting a Random Sample

In this step of the data preparation, I will use the [Dask DataFrame](https://docs.dask.org/en/stable/dataframe.html) module to read the Loan Application Register file and select a random sample to work with it in the following sections.

In [3]:
# Path to the LAR .txt file
lar_file_path = "./data/2022_combined_mlar_header.txt"

# List of values considered as missing values
missing_vals = ["Exempt", "NA", "N/A", "#NA", None, "1111", "8888", "9999"]

# Read the LAR file into a dask dataframe
ddf = dd.read_csv(lar_file_path, sep="|", dtype=ls.lar_dask_dtypes, na_values=missing_vals)

# Select a random sample, reset and drop the previous index, and compute the results
lar = ddf.sample(frac=0.015).reset_index(drop=True).compute()

### Filtering Out Missing Data

In this section I will remove columns with missing values to have a better consistency in the data.

In [4]:
# Store number of rows and columns from DataFrame shape
num_rows, num_cols = lar.shape

# Drop rows that are all missing values, modify the original dataframe
lar.dropna(how="all", inplace=True)

# Drop columns with > 90% of missing values, modify the original dataframe
lar.dropna(thresh=num_rows*0.1, axis=1, inplace=True)

# Reset and drop previous index, modify the original dataframe
lar.reset_index(drop=True, inplace=True)

### Replacing Values

Some qualitative variables of the Loan Application Register have integers as values, as can be seen below:

In [5]:
lar[["loan_type","action_taken","purchaser_type"]].head(5)

,loan_type,action_taken,purchaser_type
0,1,4,0
1,3,1,2
2,1,3,0
3,1,1,1
4,1,6,1


The `assign_labels` function will help on having descriptive values for the qualitative columns:

In [6]:
def assign_labels(dict_mapper: dict, data_frame: pd.DataFrame):
    """Assigns labels to the given pandas DataFrame columns 
    based on a dictionary that maps values to labels."""
    cols = [col for col in dict_mapper.keys() if col in data_frame.columns]
    for col in cols:
        data_frame[col] = data_frame[col].map(dict_mapper[col])

In [7]:
# Assign labels to qualitative columns
assign_labels(ls.labels_to_values, lar)

In [8]:
lar[["loan_type","action_taken","purchaser_type"]].head(5)

,loan_type,action_taken,purchaser_type
0,Conventional,Application withdrawn by applicant,Not applicable
1,Veterans Affairs,Loan originated,Ginnie Mae
2,Conventional,Application denied,Not applicable
3,Conventional,Loan originated,Fannie Mae
4,Conventional,Purchased loan,Fannie Mae


### Merging External Data

#### Legal Entity Identifiers

The [LEI Search](https://search.gleif.org/#/search/) is a tool developed by the [Global Legal Entity Identifier Foundation](https://www.gleif.org/en) (GLEIF) to access and search the complete [Legal Entity Identifier](https://www.gleif.org/en/about-lei/introducing-the-legal-entity-identifier-lei) data pool for free. 

It contains legal entity reference data on entities participating in financial transactions all over the world. 

More details on the LEI dataset fields can be found [here](https://www.gleif.org/en/about-lei/common-data-file-format/current-versions/level-1-data-lei-cdf-3-1-format).

In [9]:
# Dictionary to map original column names to new column names
gleif_cols = {"LEI":"lei", "Entity.LegalName":"institution_name", "Entity.HeadquartersAddress.City":"institution_city", "Entity.HeadquartersAddress.Country":"institution_country", "Entity.HeadquartersAddress.PostalCode":"institution_postal_code", "Entity.EntityCreationDate":"institution_years"}

# Read Legal Entity Identifier (LEI) data pool into a dask dataframe
gleif = dd.read_csv("./data/gleif-data-pool.csv", usecols=gleif_cols.keys(), dtype={"Entity.HeadquartersAddress.PostalCode":"object"})

# Unique LEIs from LAR data
unique_leis = lar["lei"].unique()

# Subset data for LEIs matching the ones in LAR
gleif = gleif.loc[gleif["LEI"].isin(unique_leis)].reset_index(drop=True)

# Rename columns and compute the results
gleif = gleif.rename(columns=gleif_cols).compute()

The `Entity.HeadquartersAddress.PostalCode` column has mixed data types due to typos in some records. To fix this, I defined the `clean_postal_code` function to clean the values.

In [10]:
# Define a lambda function to clean the postal code values
clean_postal_code = lambda code: ''.join(filter(lambda x: x.isdigit() or x == '-', str(code)))

# Applies a lambda function to slice postal codes after "-" character
gleif["institution_postal_code"] = gleif["institution_postal_code"].apply(clean_postal_code)


The `Entity.EntityCreationDate` can be used to calculate the number of years since creation of every financial institution. This is done in the below code cell:

In [11]:
# Slice creation date string without time zone and replace "nan" values with NumPy NaNs
gleif["institution_years"] = gleif["institution_years"].str[:10]
gleif["institution_years"] = gleif["institution_years"].replace("nan",np.nan)

# Fix a typo in an entity creation date, convert column to datetime
gleif.loc[gleif["institution_years"] == "1493-01-01", "institution_years"] = "1943-01-01"
gleif["institution_years"] = pd.to_datetime(gleif["institution_years"])

# Calculate the difference of each date from the 2021 year
gleif["institution_years"] = (pd.Timestamp("2021") - gleif["institution_years"]).astype("timedelta64[Y]")

Now that the `gleif` DataFrame has been cleaned, proper data types can be assigned:

In [12]:
gleif_dtypes = {"lei":"category", "institution_name":"str", "institution_city": "category", "institution_country": "category", "institution_postal_code": "category", "institution_years": "float32"}

# Assign new data types to each column
for col, dtype in gleif_dtypes.items():
    gleif[col] = gleif[col].astype(dtype)

In [13]:
# Merge GLEIF data with LAR data
lar = lar.merge(right=gleif, how="left")

#### State Names

To have more descriptive values in the data, I will merge the full name of every state code within the United States by using the [USPS State Abbreviations and FIPS Codes](https://www.bls.gov/respondents/mwr/electronic-data-interchange/appendix-d-usps-state-abbreviations-and-fips-codes.htm) data.

In [14]:
usps_abbrev_dtypes = {"state_name":"category", "state_code":"category"}

# Read the USPS State Abbreviations data
usps_abbrev = pd.read_csv("./data/us_state_codes.csv", usecols=["state_name","state_code"], dtype=usps_abbrev_dtypes)

In [15]:
# Merge USPS State Names data with LAR data
lar = lar.merge(right=usps_abbrev, how="left")

#### Gazetteer Files

The [2021 National Counties Gazetteer](https://www2.census.gov/geo/docs/maps-data/data/gazetteer/2021_Gazetteer/2021_Gaz_counties_national.zip) and [ZIP Code Tabulation Areas](https://www2.census.gov/geo/docs/maps-data/data/gazetteer/2022_Gazetteer/2022_Gaz_zcta_national.zip) data will be used to retrieve latitudes and longitudes by county code and by financial institution postal code, respectively.

This data is provided by the [United States Census Bureau](https://www.census.gov/data.html) in the [Gazetteer Files](https://www.census.gov/geographies/reference-files/time-series/geo/gazetteer-files.2021.html) web page.

> **Note**: Some Gazetteer `.txt` files may have excessive blank spaces in the header names. The following functions will fix this.

In [16]:
def downcast_numeric(data_frame: pd.DataFrame) -> pd.DataFrame:
    """Convert 64-bit numeric data types to 32-bit"""
    # Convert all int64 columns to int32
    int_cols = data_frame.select_dtypes(include=["int64"]).columns
    data_frame[int_cols] = data_frame[int_cols].astype("int32")

    # Convert all float64 columns to float32
    float_cols = data_frame.select_dtypes(include=["float64"]).columns
    data_frame[float_cols] = data_frame[float_cols].astype("float32")

    return data_frame

def clean_gazetteer(file_path: str, selected_cols: list, data_type: dict) -> pd.DataFrame:
    """Reads U.S. Gazetteer Files and returns a cleaned DataFrame"""
    # Read Gazetteer .txt file
    df = pd.read_csv(file_path, delimiter="\t", dtype=data_type)
    
    # Rename columns without blank spaces
    df = df.rename(columns={col:col.strip() for col in df.columns})

    # Downcast numeric columns to reduce memory usage
    df = downcast_numeric(df)

    return df[selected_cols]

In [17]:
# National Counties Gazetteer
gazc_names = {"GEOID":"county_code", "NAME":"county_name", "INTPTLAT":"county_lat", "INTPTLONG":"county_long"}
gaz_counties = clean_gazetteer("./data/2021_Gaz_counties_national.txt", gazc_names.keys(), data_type={"GEOID":"category"})
gaz_counties = gaz_counties.rename(columns=gazc_names)

In [19]:
# Merge National Counties Gazetteer data with LAR data
lar = lar.merge(right=gaz_counties, how="left")

In [20]:
# ZIP Code Tabulation Areas
gaz_zips_names = {"GEOID":"institution_postal_code", "INTPTLAT":"institution_lat", "INTPTLONG":"institution_long"}
gaz_zips = clean_gazetteer("./data/2022_Gaz_zcta_national.txt", gaz_zips_names.keys(), data_type={"GEOID":"category"})
gaz_zips = gaz_zips.rename(columns=gaz_zips_names)

In [21]:
# Merge ZIP Code Tabulation Areas data with LAR data
lar = lar.merge(right=gaz_zips, how="left")

### Display Options

The last step before diving deep into exploratory analysis will be to modify the number of maximum columns and how large floating point numbers are displayed in `pandas`:

In [23]:
# Set the maximum number of columns to the total number of columns in the data
pd.options.display.max_columns = lar.shape[1]

# Set the float format to be represented with commas per thousand and avoid scientific notation
pd.options.display.float_format = lambda x: '{:,.2f}'.format(x) if abs(x) < 1000000 else '{:,.0f}'.format(x)

## Data Preview

The firs step in this section is to understand the frequency of missing values within the LAR data.

In [26]:
def missing_freq_table(data_frame: pd.DataFrame) -> pd.DataFrame:
    """Creates a frequency table of missing data for the given pandas DataFrame."""
    # Freequency table of missing values per column
    nas = data_frame.isna().sum().sort_values(ascending=False)

    # Filter out columns with non-missing values
    nas = nas[nas > 0].to_frame(name="counts")

    # Add a relative frequency column
    nas["proportion"] = nas["counts"] / len(data_frame)

    # Add a percentage column
    nas["percentage"] = nas["proportion"] * 100

    # Add a data type column
    nas["data_type"] = data_frame[nas.index].dtypes.astype(str)

    return nas.reset_index().rename(columns={"index": "column_name"})

In [28]:
# Create a frequency table of missing values
na_freqs = missing_freq_table(lar)

In [30]:
# Bar plot of missing values
nas_bar = px.bar(
    na_freqs, 
    y="column_name",
    x="percentage",
    log_x=True,
    color="data_type",
    title="Percentage of Missing Values by Column"
)

nas_bar.update_layout(
    legend_title="Data Type",
    yaxis_title="Column Name",
    xaxis_ticksuffix = "%",
    xaxis_title="% of Missing Values"
)

# Arrange y-axis in ascending order
nas_bar.update_yaxes(categoryorder="total ascending")

nas_bar.show()

In [31]:
lar.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 241152 entries, 0 to 241151
Data columns (total 70 columns):
 #   Column                                    Non-Null Count   Dtype   
---  ------                                    --------------   -----   
 0   activity_year                             241152 non-null  float32 
 1   lei                                       241152 non-null  object  
 2   loan_type                                 241152 non-null  category
 3   loan_purpose                              241152 non-null  category
 4   preapproval                               241152 non-null  category
 5   construction_method                       241152 non-null  category
 6   occupancy_type                            241152 non-null  category
 7   loan_amount                               241152 non-null  float32 
 8   action_taken                              241152 non-null  category
 9   state_code                                238397 non-null  object  
 10  county_c

In [32]:
lar.head(5)

,activity_year,lei,loan_type,loan_purpose,preapproval,construction_method,occupancy_type,loan_amount,action_taken,state_code,county_code,census_tract,applicant_ethnicity_1,co_applicant_ethnicity_1,applicant_ethnicity_observed,co_applicant_ethnicity_observed,applicant_race_1,co_applicant_race_1,applicant_race_observed,co_applicant_race_observed,applicant_sex,co_applicant_sex,applicant_sex_observed,co_applicant_sex_observed,applicant_age,applicant_age_above_62,co_applicant_age,co_applicant_age_above_62,income,purchaser_type,rate_spread,hoepa_status,lien_status,applicant_credit_scoring_model,co_applicant_credit_scoring_model,denial_reason_1,total_loan_costs,origination_charges,discount_points,lender_credits,interest_rate,debt_to_income_ratio,combined_loan_to_value_ratio,loan_term,intro_rate_period,balloon_payment,interest_only_payment,negative_amortization,other_non_amortizing_features,property_value,manufactured_home_secured_property_type,manufactured_home_land_property_interest,total_units,submission_of_application,initially_payable_to_institution,aus_1,reverse_mortgage,open_end_line_of_credit,business_or_commercial_purpose,institution_name,institution_city,institution_country,institution_postal_code,institution_years,state_name,county_name,county_lat,county_long,institution_lat,institution_long
0,"2,022.00",549300LBCBNR1OT00651,Conventional,Cash-out refinancing,Preapproval not requested,Site-built,Principal residence,"165,000.00",Application withdrawn by applicant,RI,44007,44007011901,Not Hispanic or Latino,Not Hispanic or Latino,Not collected on the basis of visual observati...,Not collected on the basis of visual observati...,White,White,Not collected on the basis of visual observati...,Not collected on the basis of visual observati...,Male,Female,Not collected on the basis of visual observati...,Not collected on the basis of visual observati...,35-44,No,35-44,No,78.00,Not applicable,NaN,Not applicable,Secured by a first lien,Not applicable,Not applicable,Not applicable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,360.00,NaN,No balloon payment,No interest-only payments,No negative amortization,No other non-fully amortizing features,NaN,Not applicable,Not applicable,1,Submitted directly to your institution,Initially payable to your institution,Desktop Underwriter (DU),Not a reverse mortgage,Not an open-end line of credit,Not primarily for a business or commercial pur...,NATIONSTAR MORTGAGE LLC,Coppell,US,75019,NaN,Rhode Island,Providence County,41.87,-71.58,32.96,-96.99
1,"2,022.00",549300DD5QQUHO6PCH70,Veterans Affairs,Home purchase,Preapproval not requested,Site-built,Principal residence,"425,000.00",Loan originated,MS,28073,28073020206,Information not provided by applicant in mail ...,NaN,Not collected on the basis of visual observati...,Not collected on the basis of visual observati...,Information not provided by applicant in mail ...,Information not provided by applicant in mail ...,Not collected on the basis of visual observati...,Not collected on the basis of visual observati...,Information not provided by applicant in mail ...,Information not provided by applicant in mail ...,Not collected on the basis of visual observati...,Not collected on the basis of visual observati...,35-44,No,45-54,No,148.00,Ginnie Mae,-0.16,Not a high-cost mortgage,Secured by a first lien,Equifax Beacon 5.0,FICO Risk Score Classic 04,Not applicable,"2,528.00",0.00,264.74,NaN,6.88,50%-60%,NaN,360.00,NaN,No balloon payment,No interest-only payments,No negative amortization,No other non-fully amortizing features,"435,000.00",Not applicable,Not applicable,1,Submitted directly to your institution,Initially payable to your institution,Desktop Underwriter (DU),Not a reverse mortgage,Not an open-end line of credit,Not primarily for a business or commercial pur...,"Mortgage Research Center, LLC",Columbia,US,65203,4.00,Mississippi,Lamar County,31.20,-89.51,38.90,-92.40
2,"2,022.00",549300LBCBNR1OT00651,Conventional,Home purchase,Preapproval not requested,Site-built,Inve

## Exploratory Data Analysis

At this point, the data is ready to be summarized and visualized. This will be done by exploring every subject of study in the following order: the loan, the applicant, the lender, and the property.

### Custom Functions

The below functions are build to prevent repeating code during the analysis:

In [33]:
def destimate(data_frame: pd.DataFrame, qual_feat: list, quant_feat: str) -> pd.DataFrame:
    """Computes estimates of frequency, location, variability, and distribution
    of a quantitative feature by each category of a qualitative feature."""
    # List of estimate functions
    statistics = ["count", lambda x: x.count() / data_frame[quant_feat].count(),
                  "sum", "mean", lambda x: trim_mean(x, 0.1), "min", 
                  lambda x: x.quantile(0.25), "median", 
                  lambda x: x.quantile(0.75), "max", lambda x: x.max() - x.min(), 
                  lambda x: x.quantile(0.75) - x.quantile(0.25), "std", "var", "skew", 
                  lambda x: kurtosis(x)]
    
    # Column names for lambda functions
    lambda_names = {"<lambda_0>":"prop", "<lambda_1>":"trimean","<lambda_2>":"q1", 
                   "<lambda_3>":"q3", "<lambda_4>":"range", "<lambda_5>":"iq_range", 
                   "<lambda_6>":"kurt"}
    
    # Group by given cat_col, aggregate descriptive statistics
    agg_df = data_frame.groupby(qual_feat)[quant_feat].agg(statistics).reset_index()
    
    # Rename lambda functions
    agg_df = agg_df.rename(columns=lambda_names)
    
    # Sort by counts in descending order
    agg_df = agg_df.sort_values(by="count", ascending=False).reset_index(drop=True)

    return agg_df[agg_df["count"] > 0] #Return results with non-zero counts

### The Loan

Let's start by analizing all quantitative features related to the loan itself.

In [34]:
# List of all quantitative features related to the loan itself
loan_quant_feats = ["loan_amount","interest_rate","rate_spread","total_loan_costs","origination_charges","discount_points","lender_credits","loan_term"]

#### Loan Amount

In [35]:
lar[loan_quant_feats[0]].describe()

count    241,152.00
mean     319,130.78
std       2,239,962
min        5,000.00
25%      115,000.00
50%      225,000.00
75%      365,000.00
max     725,004,992
Name: loan_amount, dtype: float64

## Resources
- [2022 HMDA Documentation](https://ffiec.cfpb.gov/documentation/2022)
- [2022 Modified LAR Schema](https://ffiec.cfpb.gov/documentation/2022/modified-lar-schema/)
- [Using Modified LAR Data](https://github.com/cfpb/hmda-platform/blob/master/docs/UsingModifiedLar.md)
- [2022 HMDA Data on Mortgage Lending Now Available](https://www.consumerfinance.gov/about-us/newsroom/2022-hmda-data-on-mortgage-lending-now-available/)